# CHAPTER5　ネットワーキング・コンテンツ配信

## 5-1　仮想ネットワーク

**VPC**（Virtual Private Cloud）：AWS上にプライベートネットワーク空間を構築する機能<br>
**サブネット**：VPCをさらに分割したアドレス空間<br>
**ルートテーブル**：宛先によって通信手段を定義する機能<br>
**VPCエンドポイント**:インターネットを介さず、AWSのサービスとセキュアに通信することができる機能

サブネットは、VPC内のアベイラビリティーゾーンごとに作成する必要がある。

VPCの作成時には、2つの注意点がある。

・ネットワークアドレスを重複させない<br>
・最低でも/28のプレフィックス長を使用する<br>
・ロードバランサー（ELB/ALB/NLB）を配置するサブネットは、IPアドレスに余裕を持たせておく

サブネットには、次の2種類がある。

・パブリックサブネット：ルートテーブルにIGW（インターネットゲートウェイ）が**紐づいている**<br>
・プライベートサブネット：ルートテーブルにIGW（インターネットゲートウェイ）が**紐づいていない**

インターネットやプライベートサブネット内には、以下を用いてアクセスを行う。

・NATゲートウェイ：プライベートサブネットから**インターネット**にアクセスするために、**パブリックサブネット**に配置する<br>
・踏み台：プライベートサブネットの**インスタンスやデータベース**にアクセスするために、**パブリックサブネット**に配置する

NATゲートウェイには、2つの注意点がある。

・アベイラビリティゾーンごとに、1つのNATゲートウェイの配置が必要<br>
・インターネット側からNATゲートウェイにはアクセスできない

## 5-2　ネットワークセキュリティ

AWSには、ネットワークセキュリティを向上させる機能として、以下の2つがある。

セキュリティグループ：**プロトコル、ポート、IPアドレス**の3つを設定してアクセス制限を行う、**インバウンドとアウトバウンド**のアクセス制御が可能<br>
ネットワークACL：**サブネット**に対してアクセス制御を行う

尚、2つには以下のような違いがある。

セキュリティグループ → **ENI**（Elastic Network Interface）にアタッチし、**許可のみ**可能で、**ステートフル**である<br>
ネットワークACL → **サブネット**にアタッチし、**許可と拒否**が可能で、**ステートレス**である

ステートフル：行きの通信が許可されていれば、戻りの通信も自動的に許可される<br>
ステートレス：行きの通信と戻りの通信は、別々に設定をできてしまう<br>

オススメなのは、ネットワークACLでサブネットレベルに広く制限をかけて<br>
セキュリティグループでインスタンスごとに細かく制限をかけることである。

## 5-3　プライベート通信

AWSにはVPC以外のネットワーク（自社のネットワークなど）と、セキュアに通信するための機能が提供されている。

・**Direct Connect**：オンプレミス（自社のネットワークなど）とVPCを専用線で接続するサービス<br>
・**VPN**：バーチャルプライベートゲートウェイを利用した、サイト間をつなぐネットワーク<br>
・**VPCピアリング**：VPC同士をセキュアに通信できるサービス

## 5-4　DNS

・DNS（Domain Name System）：**IPアドレス**と**ドメイン名**（xxx.co.jpやxxx.comなど）を紐づける仕組み<br>
・IPアドレス：インターネットにおける住所、アクセス先<br>
・ドメイン名：分かりやすい名前（社名などを使うことが多い）

本来、サイトなどの住所はドメイン名ではなく、192.41.192.129などのIPアドレスとなっている。<br>
これを人が分かりやすいドメイン名（xxx.co.jpやxxx.comなど）に変換するシステムがDNSである。

EC2やRDS、ALBなどを起動した際のサーバーごとの接続ポイントは、すべてAWSが用意したDNS名で接続できる。<br>
つまり、各サービスにおけるIPアドレスを、分かりやすい名前と紐づけて接続できるようにしている。

Route 53の機能には、以下のものがある。

・シンプルルーティング：従来のDNSと同様のサービス<br>
・加重ルーティング：**複数**のエンドポイントに対してトラフィックを分散させることができる<br>
・レイテンシールーティング：複数リージョンでサービスを展開している際に、**遅延が一番小さい**リージョンにルーティングする<br>
・フェイルオーバールーティング：エンドポイントを監視し、**疎通が取れなくなった場合**に、別のエンドポイントにルーティングする<br>
・位置情報ルーティング：クライアントの**位置情報**をもとに、エンドポイントを変える

## 5-5　CDN

・CDN（Content Delivery System）：世界中に分散配置されたサーバー群にコンテンツを配信し、高速なコンテンツ配信を可能にするもの<br>
・**CloudFront**：AWSのDNSサービス

CloudFrontでは、アクセスの多い静的コンテンツへのアクセスはS3、動的コンテンツのアクセスはELB・EC2に向けることができる。<br>

・静的コンテンツ：トップページやニュースリリース<br>
・動的コンテンツ：お問い合わせフォーム など

こうすることで、負荷の高いアクセス（静的コンテンツ）をCloudFront+S3で低コストに捌きながら<br>
負荷の少ないアクセス（動的コンテンツ）をEC2の安いインスタンスタイプで構成するような最適化を行うことができる。